In [17]:
import pandas as pd
from coinbase.wallet.client import Client
from sec import api_key, api_sec

client = Client(api_key, api_sec)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [18]:
def get_paginated_accounts (limit):
    #annab valja listi koikide accountidega
    all_items = []
    starting_after = None
    while True:
        items = client.get_accounts(limit = limit, starting_after = starting_after)
        if items.pagination.next_starting_after is not None:
            starting_after = items.pagination.next_starting_after
            all_items += items.data
        else:
            all_items += items.data
            break
    return all_items

In [19]:
def get_transaction_table_sorted(accounts):
    all_transactions = pd.DataFrame()

    for account in accounts:
        if len(account['id']) < 10:
            continue
        print('checking ', account.balance.currency)
        local_transactions = pd.DataFrame()
        starting_after = None
        while True:
            transactions = client.get_transactions(account['id'], limit = 100, starting_after = starting_after)
            if transactions.pagination.next_starting_after is not None:
                starting_after = transactions.pagination.next_starting_after
                local_transactions = pd.concat([local_transactions, pd.DataFrame(transactions.data)])
            else:
                local_transactions = pd.concat([local_transactions, pd.DataFrame(transactions.data)])
                break
        if local_transactions.empty:
            continue

        
        local_transactions = local_transactions.sort_values('created_at')
        all_transactions = pd.concat([all_transactions, local_transactions])
    all_transactions.loc[:,'details'] = all_transactions.loc[:,'details'].apply(lambda x: x.get('header'))
    all_transactions.loc[:,'currency'] = all_transactions.loc[:,'amount'].apply(lambda x: x.get('currency'))
    all_transactions.loc[:,'crypto_amount'] = all_transactions.loc[:,'amount'].apply(lambda x: x.get('amount'))
    all_transactions.loc[:,'native_currency'] = all_transactions.loc[:,'native_amount'].apply(lambda x: x.get('currency'))
    all_transactions.loc[:,'native_amount'] = all_transactions.loc[:,'native_amount'].apply(lambda x: x.get('amount'))
    all_transactions['native_amount'] = all_transactions['native_amount'].apply(lambda x: float(x))
    all_transactions['crypto_amount'] = all_transactions['crypto_amount'].apply(lambda x: float(x))
    all_transactions = all_transactions[['id','native_amount','created_at','type','details','currency','crypto_amount','native_currency']]
    return all_transactions

In [20]:
def get_fifo_table(tabel_of_a_crypto):    
    tabel_of_sales_of_a_buy = pd.DataFrame(columns=['buy_id', 'currency', 'buy_amount_to_this_sell', 'buy_amount_in_native', 'sale_amount_in_crypto', 'sale_amount_in_native', 'buy_ppc', 'sell_ppc', 'profit in %', 'tax_owned_for_this_transaction', 'index_of_sale'])
    tabel_of_a_crypto.loc[:,'to_be_spent_from_this_buy'] = tabel_of_a_crypto['crypto_amount'][tabel_of_a_crypto['crypto_amount'] > 0]
    sales_locations = tabel_of_a_crypto[tabel_of_a_crypto['crypto_amount'] < 0].index
    for sale_index in sales_locations:
        sale = tabel_of_a_crypto.loc[sale_index]
        sale_size = abs(sale.crypto_amount)
        while sale_size > sum(tabel_of_sales_of_a_buy[tabel_of_sales_of_a_buy['index_of_sale'] == sale_index]['buy_amount_to_this_sell']):
            already_spent_on_this_sell = sum(tabel_of_sales_of_a_buy[tabel_of_sales_of_a_buy['index_of_sale'] == sale_index]['buy_amount_to_this_sell'])
            try:
                use_a_new_buy = tabel_of_a_crypto[tabel_of_a_crypto['to_be_spent_from_this_buy'] > 0].iloc[0]
            except:
                break
            buy_ppc = use_a_new_buy['native_amount'] / use_a_new_buy['crypto_amount']
            sell_ppc = sale['native_amount'] / sale['crypto_amount']
            if (use_a_new_buy['to_be_spent_from_this_buy'] + already_spent_on_this_sell) > sale_size:
                spend_from_this_buy = sale_size - already_spent_on_this_sell
                index = tabel_of_a_crypto[tabel_of_a_crypto['id'] == use_a_new_buy['id']].index
                tabel_of_a_crypto.loc[index, 'to_be_spent_from_this_buy'] = use_a_new_buy['to_be_spent_from_this_buy'] - spend_from_this_buy
                temp = (use_a_new_buy['id'], use_a_new_buy['currency'], spend_from_this_buy, use_a_new_buy['native_amount'], sale_size, sale.native_amount, buy_ppc, sell_ppc, 0, 0, sale_index)
                tabel_of_sales_of_a_buy.loc[len(tabel_of_sales_of_a_buy)] = temp
                break
            else:
                temp = (use_a_new_buy['id'], use_a_new_buy['currency'], use_a_new_buy['to_be_spent_from_this_buy'], use_a_new_buy['native_amount'], sale_size, sale.native_amount, buy_ppc, sell_ppc, 0, 0, sale_index)
                index = tabel_of_a_crypto[tabel_of_a_crypto['id'] == use_a_new_buy['id']].index
                tabel_of_a_crypto.loc[index, 'to_be_spent_from_this_buy'] = 0
                tabel_of_sales_of_a_buy.loc[len(tabel_of_sales_of_a_buy)] = temp
    tabel_of_sales_of_a_buy.loc[:, 'profit in %'] = (tabel_of_sales_of_a_buy['sell_ppc'] / tabel_of_sales_of_a_buy['buy_ppc'])*100
    tabel_of_sales_of_a_buy.loc[:, 'tax_owned_for_this_transaction'] = round(( ((tabel_of_sales_of_a_buy['buy_amount_to_this_sell'] / tabel_of_sales_of_a_buy['sale_amount_in_crypto']) * abs(tabel_of_sales_of_a_buy['sale_amount_in_native'])) - (tabel_of_sales_of_a_buy['buy_amount_to_this_sell'] * tabel_of_sales_of_a_buy['buy_ppc']) ) * 0.2, 4)
    tabel_of_sales_of_a_buy['tax_owned_for_this_transaction'][tabel_of_sales_of_a_buy['profit in %'] < 100] = 0  
    return tabel_of_sales_of_a_buy

In [21]:
def get_fifo_of_all_cryptos(all_transactions):
    total_sold = 0
    currencies = all_transactions.currency.unique()
    the_master_taxation_table = pd.DataFrame()
    for currency in currencies:   
        tabel_of_a_crypto = all_transactions[all_transactions['currency'] == currency]
        
        
        tabel_of_sales_of_a_buy = get_fifo_table(tabel_of_a_crypto)
        the_master_taxation_table = pd.concat([the_master_taxation_table, tabel_of_sales_of_a_buy])
        
        
        unique_sells = tabel_of_sales_of_a_buy.drop_duplicates('index_of_sale')
        total_sold = total_sold + abs(sum(unique_sells['sale_amount_in_native']))

    total_bought = sum(the_master_taxation_table['buy_amount_to_this_sell']*the_master_taxation_table['buy_ppc'])
    print('soetamismaksumus ', total_bought)
    print('müügi-/turuhind', total_sold)
    return the_master_taxation_table    

In [22]:
def weighted_average(tabel_of_a_crypto):
    sales_locations = []
    sales_locations = tabel_of_a_crypto[tabel_of_a_crypto['native_amount'] < 0].index
    for sale in sales_locations:
        buys_before_sell_temp = tabel_of_a_crypto.iloc[:len(tabel_of_a_crypto) - sale]
        buys_before_sell = buys_before_sell_temp[buys_before_sell_temp['native_amount'] > 0]
        average = sum(buys_before_sell['native_amount']) / sum(buys_before_sell['crypto_amount'])
        tabel_of_a_crypto.loc[sale,'average_buy_price_before_this_sell'] = average
        tabel_of_a_crypto.loc[sale, 'initial_investement_to_this_sell'] = abs(average * tabel_of_a_crypto.loc[sale, 'crypto_amount'])
        tabel_of_a_crypto.loc[sale, 'profit'] = abs(tabel_of_a_crypto.loc[sale, 'native_amount']) - tabel_of_a_crypto.loc[sale, 'initial_investement_to_this_sell']
        tabel_of_a_crypto.loc[sale, 'tax_owned_for_this_txs'] = tabel_of_a_crypto.loc[sale, 'profit'] * 0.2
    return tabel_of_a_crypto
    

In [23]:
def get_weighed_average_of_all(all_transactions):
    currencies = all_transactions.currency.unique()
    the_master_taxation_table = pd.DataFrame()

    for currency in currencies:
        tabel_of_a_crypto = all_transactions[all_transactions['currency'] == currency]
        weighted_average_of_a_crypto = weighted_average(tabel_of_a_crypto)
        the_master_taxation_table = pd.concat([the_master_taxation_table, weighted_average_of_a_crypto])
    the_master_taxation_table['tax_owned_for_this_txs'][the_master_taxation_table['tax_owned_for_this_txs'] < 0]
    sells = the_master_taxation_table[the_master_taxation_table['crypto_amount'] < 0]
    total_bought = sum(sells['crypto_amount'] * sells['average_buy_price_before_this_sell'])
    total_sold = sum(sells['native_amount'])
    print('soetamismaksumus ', abs(total_bought))
    print('müügi-/turuhind', abs(total_sold))
    return the_master_taxation_table



In [24]:
accounts = get_paginated_accounts(limit = 100)
all_transactions = get_transaction_table_sorted(accounts) #gets all transactions

checking  GFI
checking  REP
checking  CRV
checking  BADGER
checking  COMP
checking  CTSI
checking  FOX
checking  ALCX
checking  USDT
checking  DDX
checking  BLZ
checking  FX
checking  TRAC
checking  BAL
checking  IMX
checking  JASMY
checking  SNX
checking  IDEX
checking  SUSHI
checking  PAX
checking  POWR
checking  MIR
checking  BNT
checking  POLS
checking  COVAL
checking  ALGO
checking  ASM
checking  LPT
checking  DESO
checking  ANKR
checking  ENS
checking  XLM
checking  XTZ
checking  RLY
checking  OGN
checking  PLA
checking  ACH
checking  CGLD
checking  VGX
checking  RLC
checking  QUICK
checking  AGLD
checking  FET
checking  OXT
checking  ZEN
checking  MUSD
checking  NU
checking  GODS
checking  AUCTION
checking  BOND
checking  GYEN
checking  POLY
checking  SHIB
checking  DOGE
checking  RAI
checking  GALA
checking  ARPA
checking  API3
checking  ATOM
checking  LCX
checking  LRC
checking  FIL
checking  TRIBE
checking  SUKU
checking  ORN
checking  UST
checking  FARM
checking  RARI
checki

In [25]:
the_master_table = get_weighed_average_of_all(all_transactions) #gets weighed average of all transactions

soetamismaksumus  13552.218990267796
müügi-/turuhind 20230.57


/var/folders/c3/wxvrld895cx0w3cjdwz8hrkw0000gn/T/ipykernel_81070/3479669299.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabel_of_a_crypto.loc[sale,'average_buy_price_before_this_sell'] = average
/var/folders/c3/wxvrld895cx0w3cjdwz8hrkw0000gn/T/ipykernel_81070/3479669299.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabel_of_a_crypto.loc[sale, 'initial_investement_to_this_sell'] = abs(average * tabel_of_a_crypto.loc[sale, 'crypto_amount'])
/var/folders/c3/wxvrld895cx0w3cjdwz8hrkw0000gn/T/ipyker

In [26]:
the_master_table = get_fifo_of_all_cryptos(all_transactions) #gets fifo of all transactions

soetamismaksumus  18829.151380200397
müügi-/turuhind 20230.570000000003


/var/folders/c3/wxvrld895cx0w3cjdwz8hrkw0000gn/T/ipykernel_81070/2736758369.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabel_of_a_crypto.loc[:,'to_be_spent_from_this_buy'] = tabel_of_a_crypto['crypto_amount'][tabel_of_a_crypto['crypto_amount'] > 0]
/var/folders/c3/wxvrld895cx0w3cjdwz8hrkw0000gn/T/ipykernel_81070/2736758369.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabel_of_sales_of_a_buy['tax_owned_for_this_transaction'][tabel_of_sales_of_a_buy['profit in %'] < 100] = 0
/var/folders/c3/wxvrld895cx0w3cjdwz8hrkw0000gn/T/ipykernel_81070/273675836

In [27]:
the_master_table

,buy_id,currency,buy_amount_to_this_sell,buy_amount_in_native,sale_amount_in_crypto,sale_amount_in_native,buy_ppc,sell_ppc,profit in %,tax_owned_for_this_transaction,index_of_sale
0,6695b0ff-c64c-5f01-a546-5e77c504489b,AXS,8.926720,601.28,8.926720,-610.94,67.357329,68.439473,101.606573,1.9320,0
0,7caee0e0-5a02-5dfb-8ee6-14e3499f4cf8,SPELL,17641.776513,349.38,17641.776513,-220.55,0.019804,0.012502,63.126109,0.0000,0
0,a2795b47-5a64-5c8a-880a-a1a397b3874b,AVAX,3.869654,539.53,3.869654,-379.60,68.970189,98.096632,142.230482,22.5418,1
1,a2795b47-5a64-5c8a-880a-a1a397b3874b,AVAX,3.953001,539.53,3.953001,-389.18,68.970189,98.451775,142.745405,23.3082,0
0,acce91be-f597-512b-8387-ccae4933762c,SOL,4.529479,662.09,7.032204,-1044.10,146.173543,148.474067,101.573831,2.0840,0
1,e1f7a6c8-9874-54ba-a653-fa81e8c6c812,SOL,2.502725,392.03,7.032204,-1044.10,156.641235,148.474067,94.786068,0.0000,0
0,3a3e74ed-e1cc-5b9b-9ae0-88b16330044a,ADA,355.655941,870.37,355.655941,-672.05,2.447225,1.889607,77.214288,0.0000,0
0,3f9d1552-865a-5c80-9de4-51869d8fb033,DOT,1.000000,23.66,9.964501,-399.44,23.660000,40.086301,169.426460,3.2853,1
1,61359afd-f5de-5b13-a2ab-36089e9bf056,DOT,8.964501,534.76,9.964501,-399.44,23.613214,40.086301,169.762156,29.5346,1
2,61359afd-f5de-5b13-a2ab-36089e9bf056,DOT,13.682141,534.76,13.682141,-324.56,23.613214,23.721434,100.458305,0.2961,0
